In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from src.models.DCVC_net import DCVC_net
import torch
from torchvision import transforms
import numpy as np
import pathlib
import os
import matplotlib.pyplot as plt
import wandb
import tqdm

In [7]:
# BATCH_SIZE = 4
# DATA_DIR = pathlib.Path('/data2/jatin/vimeo_septuplet/sequences')
DEVICE = torch.device('cpu')
DEVICE

device(type='cpu')

In [8]:
video_net = DCVC_net()

In [9]:
# # load the good weights
# video_net.opticFlow = torch.load('../DCVC/optflow.pth')
# video_net.mvEncoder = torch.load('../DCVC/mvenc.pth')
# video_net.mvDecoder_part1 = torch.load('../DCVC/mvDecoder_part1.pth')
# video_net.mvDecoder_part2 = torch.load('../DCVC/mvDecoder_part2.pth')
# video_net.feature_extract = torch.load('../DCVC/feature_extract.pth')
# video_net.context_refine = torch.load('../DCVC/context_refine.pth')
# # video_net.contextualDecoder_part1 = torch.load('../DCVC/contextualDecoder_part1.pth')

video_net.load_state_dict(torch.load('checkpoints/model_dcvc_quality_3_psnr.pth'))
# print('Reminder to load optimizer too')
# # optimizer = torch.optim.Adam(video_net.parameters(), lr=wandb.config.learning_rate)

# video_net.opticFlow.requires_grad_ = False
# video_net.mvEncoder.requires_grad_ = False
# video_net.mvDecoder_part1.requires_grad_ = False
# video_net.mvDecoder_part2.requires_grad_ = False
# video_net.feature_extract.requires_grad_ = False
# video_net.context_refine.requires_grad_ = False
# # video_net.contextualDecoder_part1.requires_grad_ = False
video_net

DCVC_net(
  (bitEstimator_z): BitEstimator(
    (f1): Bitparm()
    (f2): Bitparm()
    (f3): Bitparm()
    (f4): Bitparm()
  )
  (bitEstimator_z_mv): BitEstimator(
    (f1): Bitparm()
    (f2): Bitparm()
    (f3): Bitparm()
    (f4): Bitparm()
  )
  (feature_extract): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ResBlock(
      (relu1): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu2): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (context_refine): Sequential(
    (0): ResBlock(
      (relu1): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu2): ReLU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (mvEncoder): Sequential(
    (0): Conv2d(2, 128, kernel_size=(3, 

In [6]:
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, crop_size=256, make_p_cut=False, make_b_cut=True, deterministic=False):
        if make_b_cut and make_p_cut:
            raise ValueError('Can only choose one of B-frames or P-frames')
        self.data_dir = data_dir
        self.crop_size = crop_size
        self.make_p_cut = make_p_cut
        self.make_b_cut = make_b_cut
        self.deterministic = deterministic
        self.all_paths = []
        for seq in os.listdir(self.data_dir):
            subseq = os.listdir(self.data_dir / seq)
            for s in subseq:
                self.all_paths.append(self.data_dir / seq / s)
        assert len(self.all_paths) == 91701
        
        self.transforms = torch.nn.Sequential(
            transforms.RandomCrop(crop_size)
        )
       
    def __getitem__(self, i):
        path = self.all_paths[i]
        imgs = []
        if self.make_b_cut:
            # load two reference frames and the B-frame in the middle
            #TODO: implement making this deterministic
            interval = np.random.randint(1, 4) # can be 1, 2, or 3
            ref1 = plt.imread(path / f'im{1}.png')
            ref2 = plt.imread(path / f'im{1 + interval*2}.png')
            # this is the B-frame, in the middle
            im = plt.imread(path / f'im{1 + interval}.png')
            imgs = [ref1, ref2, im]
        elif self.make_p_cut:
            ref = plt.imread(path / f'im1.png')
            im = plt.imread(path / f'im2.png')
            imgs = [ref, im]
        else:
            # load full sequence
            for i in range(1, 8):
                # should be between [0, 1]
                img = plt.imread(path / f'im{i}.png')
        
        # plt.imread should make inputs in [0, 1] for us
        imgs = np.stack(imgs, axis=0)
        # bring RGB channels in front
        imgs = imgs.transpose(0, 3, 1, 2)
        return self.transforms(torch.FloatTensor(imgs))

    def __len__(self):
        return len(self.all_paths)

ds = VideoDataset(DATA_DIR, make_p_cut=True, make_b_cut=False)
dl = torch.utils.data.DataLoader(
    ds,
    shuffle=True,
    batch_size=BATCH_SIZE,
    num_workers=6,
    prefetch_factor=5,
)

In [10]:
video_net = video_net.to(DEVICE)

In [11]:
def count_parameters(model):
    """Return number of parameters in a model"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(video_net)} trainable parameters')

The model has 7944448 trainable parameters


In [11]:
# def nan_to_num(x, nan=0.0, posinf=1.0, neginf=-1.0):
#     """This does not exist in PyTorch 1.7.1 so I had to write it."""
#     #TODO: make more efficient?
#     nans = torch.isnan(x)
#     posinfs = torch.isposinf(x)
#     neginfs = torch.isneginf(x)
#     x[nans] = nan
#     x[posinfs] = posinf
#     x[neginfs] = neginf
#     return x

# def handle_back_nan_hook(grad):
#     # stop nans, inf, -inf, and large values in general
#     grad = nan_to_num(grad, nan=0.0, posinf=1.0, neginf=-1.0)
#     grad = torch.clip(grad, min=-1.0, max=1.0)
#     return grad

# for p in video_net.parameters():
#     if p.requires_grad:
#         p.register_hook(handle_back_nan_hook)

In [9]:
def test_epoch(model, dl, compress):
    mse_criterion = torch.nn.MSELoss()
    model.eval()
    epoch_mse = 0
    epoch_bitrate = 0
    total_count = 0
    pbar = tqdm.tqdm(dl)
    for i, x in enumerate(pbar):
        x = x.to(DEVICE)
        ref = x[:,0]
        im = x[:,1]
        with torch.no_grad():
            preds_dict = model(ref, im, compress=compress)
        preds = preds_dict['recon_image']
        bpp = preds_dict['bpp']
        mse_loss = mse_criterion(preds, im)        
        epoch_mse += mse_loss.item()
        epoch_bitrate += bpp.item()
        total_count += 1
        if i % 1 == 0:
            avg_mse = epoch_mse / total_count
            avg_psnr = -10.0*np.log10(avg_mse)
            avg_bitrate = epoch_bitrate / total_count
            avg_mse = round(avg_mse, 6)
            avg_psnr = round(avg_psnr, 6)
            avg_bitrate = round(avg_bitrate, 6)
            pbar.set_description(f'Avg PSNR/Bitrate: {avg_psnr, avg_bitrate}')
            

In [15]:
test_epoch(video_net, dl, compress=False)

Avg PSNR/Bitrate: (41.826272, 0.057603):   4%|▍         | 1004/22926 [02:00<43:56,  8.31it/s] 


KeyboardInterrupt: 

In [10]:
test_epoch(video_net, dl, compress=True)

Avg PSNR/Bitrate: (41.423817, 0.062252):   7%|▋         | 1566/22926 [03:00<41:08,  8.65it/s]  


KeyboardInterrupt: 

In [ ]:
video_net.eval()